In [ ]:
import pandas as pd
import os
import json
import datetime as dt
from dateutil import parser 
import re
from concurrent.futures import ThreadPoolExecutor

In [ ]:
input_dir = './Sport Bettor AI input'
files = os.listdir(input_dir)
files

In [ ]:
csvs = {file.replace('.csv', ''): pd.read_csv(f'{input_dir}/{file}') \
        if file != 'Dataset#5200NBA players stats as of Nov 20th 2024 - Sheet1.csv' \
        else pd.read_csv(f'{input_dir}/{file}', header=None) for file in files if file.endswith('.csv')}

In [ ]:
csvs

In [ ]:
csvs['Dataset#5062pbp-2024'].isna().any(axis=0)

In [ ]:
ds_5265 = pd.read_csv(f'./{input_dir}/Dataset#5265expanded_overtime_markets_data.txt', delimiter='|')
ds_5265

In [ ]:
csvs['Dataset#5265expanded_overtime_markets_data'] = ds_5265

# Reformating any column with substring of date

In [ ]:
# get the parts of the date like year, month, and day and reformats it to a string
# because it is virtually impossible to reformat a date with varying format
# so year, month, and day have to be extracted first
test_dt_obj = parser.parse('1/13/15')
test_dt_obj.day, test_dt_obj.month, test_dt_obj.year

In [ ]:
reformed_date = f'{test_dt_obj.day} {test_dt_obj.month} {test_dt_obj.year}'
reformed_date

In [ ]:
new_date = dt.datetime.strptime(reformed_date, '%d %m %Y').strftime('%B %#d %Y')
new_date

In [ ]:
type(new_date)

In [ ]:
csvs['Dataset#5062pbp-2024'].columns[[0, 1]]

In [ ]:
def cohere(row):
    """
    restructures the meaningless number values in the dataframe
    and forms a coherent and meaningful sentence out of it

    args:
        row - row of a dataframe
    """
    message = ""
    df_indeces = row.keys()
    for df_index in df_indeces:
        if type(df_index) != int:
            # if header is a string specify the header in the message
            message += f"{df_index.lower().capitalize()} is {row[df_index] if not pd.isnull(row[df_index]) else "empty"}, "
        else:
            message += f"{row[df_index]} "
        

    return message

To clean the date what I can do is match anything in the Index returned by df.columns containing the string Date, date, or anything resembling it, we grab that column

In [ ]:
def cohere_csvs(csvs: dict):
    """
    creates readable messages in a list from all rows of each 
    dataframes 

    args:
        csvs - dictionary of all .csv files
        with key as filename and value as the dataframe
        read by pd.read_csv()
    """
    outputs = []
    for dataset_name, df in csvs.items():
        # REFORMAT DATE
        print(f'processing: {dataset_name}')
        # note that columns in df.columns are strings extracts the
        # indeces that contain the string date in the columsn of a df 
        date_cols = [col for col in df.columns.to_list() if "date" in str(col).lower() or "time" in str(col).lower()]
        # print(date_cols)
        
        # however many columns we preprocesses these date columns
        # such that it can be readable
        def reformat_date(date):

            datetime_obj = parser.parse(date)
            year = datetime_obj.year
            month = datetime_obj.month
            day = datetime_obj.day
            combined_date = f'{day} {month} {year}'
            reformed_date = dt.datetime.strptime(combined_date, '%d %m %Y').strftime('%B %#d %Y')
            
            return reformed_date

        # apply reformating function on all columns with date
        for date_col in date_cols:
            df[date_col] = df[date_col].apply(reformat_date)

        # REMOVES any column that has substring or resembling an `id` or `unnamed: <int>` columns
        cols_to_del = [col for col in df.columns.to_list() if bool(re.search(r"id|Id|ID$", str(col))) or bool(re.search(r"[Uu]nnamed:\s[0-9]*", str(col)))]
        df.drop(columns=cols_to_del, inplace=True)

        # COHERE ALL COLUMNS INTO ONE COHERENT MESSAGE
        df['message'] = df.apply(cohere, axis=1)

        # Populate and save the generated messages
        outputs.append((dataset_name, df['message'].to_list()))

    return outputs

In [ ]:
def clean_and_split_data(name, data, output_dir: str, char_limit: int=700, max_rows: int=40):
    """
    Cleans and splits text data:
    - Removes numbering at the start of lines.
    - Converts text to lowercase.
    - Splits each row to ensure it doesn't exceed the character limit.
    - Outputs multiple files if the number of rows exceeds max_rows.
    """
    def split_into_chunks(text, limit):
        """
        Splits a text into chunks no larger than the character limit.
        

        e.g "  "description": "Oracle-X is a...knowledge."," is a string
        or line that may ocntain 700+ characters 
        """
        
        chunks = []

        # if the length of line is bigger than limit i.e. 900 > 700
        # then the line is sliced into 700 character strings
        # and the next slice i.e. 700:900 or [700] to [899] is now set
        # as the next string to be processed but since slice [700] to [899]
        # length of 200 is now less than limit of 700 loop is terminated
        # and the final 200 characters are appended to the chunks list
        while len(text) > limit:
            chunks.append(text[:limit].strip())
            text = text[limit:]
        if text:
            chunks.append(text.strip())

        # chunks = [<chunk 1 of 700 char string>, <chunk 2 of 700 char string>, ..., <chunk n of <=700 char string>]
        return chunks

    # Process each row to enforce character limits
    output_lines = []
    for line in data:
        line = line.strip()
        output_lines.extend(split_into_chunks(line, char_limit))

    # Write to multiple files if necessary
    base_name = name

    # if output already exists use that folder (meaning don't overwrite)
    os.makedirs(output_dir, exist_ok=True)

    for i in range(0, len(output_lines), max_rows):
        """
        if there were 100 output lines and we wanted only
        40 lines per file, we would increment from
        0:0+40 or [0] to [39]
        40:40+40 or [40] to [79]
        80:80+40 or [80] to [119] 
        
        but in slicing arrays 
        when an array is only of a certain length and our end 
        index exceeds it we only really get the slice until the end of the array 
        so in essence we get only [80] to [99]
        """
        chunk = output_lines[i:i + max_rows]

        """40 is the max amount of rows"""
        output_file = os.path.join(output_dir, f"{base_name}_processed_part{i // max_rows + 1}.txt")
        print(f"output file: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.writelines(line + '\n' for line in chunk)

        print(f"File {output_file} has been created with {len(chunk)} lines.")

# Preprocessing .csv's

In [ ]:
outputs = cohere_csvs(csvs)
outputs

In [ ]:
output_dir = './Sport Bettor AI'
for name, lists in outputs:
    clean_and_split_data(name, lists, output_dir)

# New method for preprocessing .json files with nested dictionaries inside using recursion

In [327]:
# Using recursion to get all values from nested dictionary
def extract_keys_values(file: dict):
    """
    extracts all information in nested dictionaries including
    key and value pair, recursively
    """
    try:    
        non_dict_vals = []

        for key, value in file.items():
            if isinstance(value, dict):
                non_dict_vals.extend(extract_keys_values(value))

            # this is if the value of corresponding key is an iterable
            # that may or may not contain purely single values or iterables
            # or another nested dictionary
            elif isinstance(value, list):
                for i in value:
                    if isinstance(i, dict):
                        non_dict_vals.extend(extract_keys_values(i))
                    else:
                        non_dict_vals.append(f'{key} is {i}')

            # this is if value of correspoinding key is purely an int, str, date,
            # or any value that is a non iterable
            else:
                non_dict_vals.append(f'{key} is {value}')

        return non_dict_vals
    except AttributeError:
        print(file)

In [ ]:
def read_json_files(input_dir: str, files: list[str]):
    def helper(file_name):
        with open(f'{input_dir}/{file_name}', 'r') as file:
            data = json.load(file)
            file.close()

        new_name = file_name.replace('.json', '')
        return new_name, data

    # concurrently read and load all .json files
    with ThreadPoolExecutor() as exe:
        jsons = dict(list(exe.map(helper, files)))
    return jsons

In [ ]:
json_files = [file for file in files if file.endswith('.json')]
jsons = read_json_files(input_dir=input_dir, files=json_files)
jsons

In [322]:
jsons['Dataset#5210Game data']

{'description': 'Access extended information about upcoming sports games, including odds, teams, times, bet types, and more. Data is structured and identified for seamless interpretation.',
 'overtimeLink': 'https://overtimemarketsv2.xyz/overtime-v2/networks/10/markets/?status=open&ungroup=true&onlyBasicProperties=true&onlyMainMarkets=true&includeProofs=false&minMaturity=1731618761',
 'gameExample': {'gameId': '0x4341353836444533304343380000000000000000000000000000000000000000',
  'subLeagueId': 10,
  'typeId': 0,
  'line': 0,
  'maturity': 1732419000,
  'homeTeam': 'Los Angeles FC',
  'awayTeam': 'Seattle Sounders FC',
  'status': 0,
  'odds': [0.534759358289, 0.263157894737, 0.285714285714],
  'childMarkets': [{'typeId': 10001,
    'line': -0.5,
    'status': 0,
    'odds': [0.545553737043, 0.534759358289]},
   {'typeId': 10002,
    'line': 2.5,
    'status': 0,
    'odds': [0.565291124929, 0.523834468308]}],
  'statusCode': 'open'},
 'identifiers': {'gameId': 'Identifier of the game

# Note this json data actually comes first wrapped in a list so extract only the dictionary object itself before passing it through `extract_keys_values()`

In [321]:
jsons['Dataset#5212Example instructions']

[{'overtimeLink': 'https://overtimemarketsv2.xyz/overtime-v2/networks/10/markets/?status=open&ungroup=true&onlyBasicProperties=true&onlyMainMarkets=true&includeProofs=false&minMaturity=1731618761',
  'instruction': 'Interpret the following game information and respond appropriately:',
  'input': {'gameId': '0x4341353836444533304343380000000000000000000000000000000000000000',
   'subLeagueId': 10,
   'typeId': 0,
   'line': 0,
   'maturity': 1732419000,
   'homeTeam': 'Los Angeles FC',
   'awayTeam': 'Seattle Sounders FC',
   'status': 0,
   'odds': [0.534759358289, 0.263157894737, 0.285714285714],
   'childMarkets': [{'typeId': 10001,
     'line': -0.5,
     'status': 0,
     'odds': [0.545553737043, 0.534759358289]},
    {'typeId': 10002,
     'line': 2.5,
     'status': 0,
     'odds': [0.565291124929, 0.523834468308]}]},
  'output': 'This game features Los Angeles FC vs Seattle Sounders FC. The moneyline odds are 0.53 for Los Angeles FC, 0.26 for Seattle Sounders FC, and 0.29 for a 

In [324]:
jsons['Dataset#5212Example instructions'] = jsons['Dataset#5212Example instructions'][0]

In [330]:
outputs = []
for dataset_name, json in jsons.items():
    outputs.append(
        (dataset_name, extract_keys_values(json))
    )

In [331]:
outputs

[('Dataset#5210Game data',
  ['description is Access extended information about upcoming sports games, including odds, teams, times, bet types, and more. Data is structured and identified for seamless interpretation.',
   'overtimeLink is https://overtimemarketsv2.xyz/overtime-v2/networks/10/markets/?status=open&ungroup=true&onlyBasicProperties=true&onlyMainMarkets=true&includeProofs=false&minMaturity=1731618761',
   'gameId is 0x4341353836444533304343380000000000000000000000000000000000000000',
   'subLeagueId is 10',
   'typeId is 0',
   'line is 0',
   'maturity is 1732419000',
   'homeTeam is Los Angeles FC',
   'awayTeam is Seattle Sounders FC',
   'status is 0',
   'odds is 0.534759358289',
   'odds is 0.263157894737',
   'odds is 0.285714285714',
   'typeId is 10001',
   'line is -0.5',
   'status is 0',
   'odds is 0.545553737043',
   'odds is 0.534759358289',
   'typeId is 10002',
   'line is 2.5',
   'status is 0',
   'odds is 0.565291124929',
   'odds is 0.523834468308',
   

In [332]:
output_dir = './Sport Bettor AI'
for name, lists in outputs:
    clean_and_split_data(name, lists, output_dir)

output file: ./Sport Bettor AI\Dataset#5210Game data_processed_part1.txt
File ./Sport Bettor AI\Dataset#5210Game data_processed_part1.txt has been created with 40 lines.
output file: ./Sport Bettor AI\Dataset#5210Game data_processed_part2.txt
File ./Sport Bettor AI\Dataset#5210Game data_processed_part2.txt has been created with 21 lines.
output file: ./Sport Bettor AI\Dataset#5212Example instructions_processed_part1.txt
File ./Sport Bettor AI\Dataset#5212Example instructions_processed_part1.txt has been created with 24 lines.
output file: ./Sport Bettor AI\Dataset#5213Overtime data interpretation_processed_part1.txt
File ./Sport Bettor AI\Dataset#5213Overtime data interpretation_processed_part1.txt has been created with 33 lines.
output file: ./Sport Bettor AI\Dataset#6809sports_betting_accounts_processed_part1.txt
File ./Sport Bettor AI\Dataset#6809sports_betting_accounts_processed_part1.txt has been created with 40 lines.
output file: ./Sport Bettor AI\Dataset#6809sports_betting_acco